In [647]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [648]:
from src.driver.service import DriverService
from src.state_manager.state import StateManager
from src.actions.browser_actions import BrowserActions
from src.agent_interface.planing_agent import PlaningAgent

In [649]:
task = 'Go to the url kayak.com and then find a flight from Zürich to Bali on 2025-04-25 with return on 2025-06-05 for 2 people.'

In [650]:
# Start with blank.
# let llm decide decide the search action from the set of possible actions
# execute the action to search the daraz.pk
# get that page
# process the page to get index mappings of only interactable elements, texts and leaf elements containing text
# give this to llm to select new action. 
# It will specify the element to click/input. you will use the index to get the location of that element and actually clicks it

In [651]:
driver = DriverService().get_driver()
state_manager = StateManager(driver)
actions = BrowserActions(driver)

In [652]:
agent = PlaningAgent(task=task, default_actions=actions.get_default_actions(), model="gpt-4o")

In [653]:
url_history = []

---

In [852]:
current_state = state_manager.get_current_state()

In [853]:
url_history.append(driver.current_url)
text = f'Elements: {current_state.interactable_elements}, Url history: {url_history}'

In [854]:
print(text)

Elements: 2:<div aria-label="Open main navigation"></div>
3:<a href="/" aria-label="Go to the kayak homepage"></a>
4:<div aria-label="Open Trips drawer"></div>
5:<div>Sign in</div>
6:<span>Sign in</span>
372:<div aria-haspopup="listbox" aria-expanded="false" aria-label="Trip type Round-trip">Round-trip</div>
373:<span>Round-trip</span>
375:<div>0 bags</div>
376:<span>0 bags</span>
378:<div>ISB</div>
380:<div aria-label="Remove value"></div>
381:<div>ZRH</div>
383:<div aria-label="Remove value"></div>
384:<input aria-readonly="false" aria-label="Flight origin input" aria-autocomplete="list" aria-haspopup="listbox" aria-controls="flight-origin-smarty-input-list"></input>
385:<div aria-label="Swap departure airport and destination airport"></div>
386:<div>Denpasar, Bali, Indonesia (DPS)</div>
388:<div aria-label="Remove value"></div>
390:<div aria-label="Departure">Fri 4/25</div>
391:<span>Fri 4/25</span>
393:<div aria-hidden="true">–</div>
395:<div aria-label="Return">Thu 6/5</div>
396:<

In [855]:
action = agent.chat(text)

Total prompt cost:  $0.01 Total tokens:  2,390


In [856]:
input_ = "\n\n".join([f"{msg['role']}:\n{msg['content']}" for msg in agent.input_messages])
print(input_)

system:

You are an AI agent that helps users navigate websites and perform actions.

## Task:
Go to the url kayak.com and then find a flight from Zürich to Bali on 2025-04-25 with return on 2025-06-05 for 2 people.

## Available actions:
At each step, you must choose an action from the predefined set of actions in the format:  
{name: arguments_definition}
The available actions are:  
{'search_google': 'text: string', 'go_to_url': 'url: string', 'done': '', 'go_back': '', 'click': 'id: int', 'input': 'id: int, text: string', 'nothing': ''}  

## Input:
The page content will be provided as numbered elements like this:
0:<button>Click me</button>
1:<a href="/test">Link text</a>
2:Some visible text content 

Additionally you get sequence of previous actions.

## Instructions:
- Provide your next action based on the available actions, previous actions and visible elements. 
- Make sure to follow action's argument_definition
- Each element has a unique index that can be used to interact wi

In [859]:
action

Action(action='done', params=None, goal='Complete and finalize the input as was successfully set to target 2 adults for travel', valuation_previous_goal='Success! The number of adults is now correctly set to 2 for the flight search.')

In [860]:
done = actions.execute_action(action, current_state.selector_map)

{
    "action": "done",
    "params": null,
    "goal": "Complete and finalize the input as was successfully set to target 2 adults for travel",
    "valuation_previous_goal": "Success! The number of adults is now correctly set to 2 for the flight search."
}


In [837]:
done

---

In [861]:
driver.quit()